<b>
<center>
<h1>Predicting Readmission and Treatments for Diabetic Patients</h1> <br>
<h2>Final Project - Machine Learning I - Spring 2020 </h2><br>
<h3>Team members: Trinh Vu, Madhuri Yadav, Sharmin Kantharia</h3>
<h3>Part 1 - Predicting Readmission</h3>
</center>
</b>





# Introduction
We will work on Kaggle dataset [Diabetes 130 US hospitals for years 1999 - 2008](https://www.kaggle.com/brandao/diabetes) 

The data represents 10 years of clinical care at 130 US hospitals. It includes 50 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria.

(1) It is an inpatient encounter (a hospital admission).

(2) It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.

(3) The length of stay was at least 1 day and at most 14 days.

(4) Laboratory tests were performed during the encounter.

(5) Medications were administered during the encounter.

The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc.

The target variables will be readmission and treatment. Our goals is to predict the readmission and treatment of a patient.

## Google Drive

In [1]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

# Set working directory
abspath = '/content/drive/My Drive/Colab Notebooks/ML1/Final Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Warning

In [0]:
import warnings
# Ignore warnings
warnings.filterwarnings('ignore')

# Data Preprocessing


## Loading data

In [35]:
import pandas as pd

# Load the raw data
df_raw = pd.read_csv(abspath + 'diabetic_data.csv', header = 0, na_values=['?'])

# Make a copy of df_raw
df = df_raw.copy(deep=True)

# Print the dimension of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns = ['# rows', '# columns'])

# rows  # columns
0  101766         50

In [4]:
# Print the first 5 rows of df
df.head()

encounter_id  patient_nbr             race  ... change diabetesMed readmitted
0       2278392      8222157        Caucasian  ...     No          No         NO
1        149190     55629189        Caucasian  ...     Ch         Yes        >30
2         64410     86047875  AfricanAmerican  ...     No         Yes         NO
3        500364     82442376        Caucasian  ...     Ch         Yes         NO
4         16680     42519267        Caucasian  ...     Ch         Yes         NO

[5 rows x 50 columns]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

## Handling the Identifiers

### Identify the Identiers

In [0]:
# Define a function to get the identifiers
def id_checker(df):
  '''
  The function checks the identifiers
  Parameter df: dataframe
  Returns the dataframe of the identifers
  '''
  df_id = df[[var for var in df.columns 
              if df[var].nunique(dropna=True) == df[var].notnull().sum()]]
  return df_id

In [37]:
# Call the function on df
df_id = id_checker(df)

# Print the first 5 rows of df_id
df_id.head()

encounter_id
0       2278392
1        149190
2         64410
3        500364
4         16680

## Separating 24 treatments into 2 categories 'insulin' and 'io' (stands for insulin + others) and create 1 column of Treatment.

### Use the identifier to separate treatments

In [0]:
# Create a dataframe contains only treatment and encounter_id
treatments = df[['encounter_id','metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']].copy()

In [0]:
# Label values from treatment as 0 or 1. 0 means the medication is not prescribed.
# 1 means the medication is prescribed. 'Steady', 'Up', 'Down' are labled as 1
treatments=treatments.replace(['No','Steady','Up','Down'],[0,1,1,1])
treatments.set_index('encounter_id',inplace=True)

In [40]:
# Print first 5 rows of treatments
treatments.head()

metformin  ...  metformin-pioglitazone
encounter_id             ...                        
2278392               0  ...                       0
149190                0  ...                       0
64410                 0  ...                       0
500364                0  ...                       0
16680                 0  ...                       0

[5 rows x 23 columns]

In [41]:
treatments.sum(axis=1).value_counts()

1    47314
0    23403
2    21873
3     7778
4     1335
5       58
6        5
dtype: int64

Patients are given combination of 6 medications (at maximum). 
We will encode the treatment so that when the patient is given only insulin, we label the treatment as 'insulin'. When patient is given insulin and other medications, we label it as 'io' (insulin + others). We label 'others' otherwise.

### Creating a new feature "Treatments"


#### 1. When the value of Insulin is '1' , create the classes "insulin" & "io" (insulin + others )

In [0]:
insulin1 = treatments[treatments['insulin']==1].sum(axis = 1).replace([1,2,3,4,5,6],['insulin','io','io','io','io','io'])

#### 2. When the value of Insuin is '0' , create the classes "others" & "no med"

In [0]:
insulin0 = treatments[treatments['insulin']==0].sum(axis=1).replace([0,1,2,3,4,5,6],['no med','other','other','other','other','other','other'])

In [0]:
# Combine insulin1 and insulin0
treatments=pd.concat([insulin1,insulin0])
treatments = pd.DataFrame({'treatments':treatments})

In [0]:
# Add treatments feature to the df dataframe
df=df.join(treatments,on='encounter_id') #setting index as encounter_id

In [0]:
# Now we have treatments as 1 column, we will remove the 24 treatments from df
df = df.drop(columns = ['metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone'])

In [0]:
# Remove 'other' class from treatments column
df = df[df.treatments!='other']

# Remove 'no med' class from treatments column
df = df[df.treatments!='no med']

### Remove the Identifier

In [0]:
# Remove the identifiers
df = df.drop(columns = 'encounter_id')

In [0]:
# Patient_nbr and any id numbers such as admission_type, admission_source, and discharge_disposition will also be removed because they don't effect the predictions of readmission or treatment.
df = df.drop(columns = ['patient_nbr', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id'])

In [50]:
# Print the first 5 rows of df
df.head()

race  gender      age weight  ...  change diabetesMed readmitted  treatments
1  Caucasian  Female  [10-20)    NaN  ...      Ch         Yes        >30     insulin
3  Caucasian    Male  [30-40)    NaN  ...      Ch         Yes         NO     insulin
4  Caucasian    Male  [40-50)    NaN  ...      Ch         Yes         NO          io
5  Caucasian    Male  [50-60)    NaN  ...      No         Yes        >30     insulin
6  Caucasian    Male  [60-70)    NaN  ...      Ch         Yes         NO          io

[5 rows x 23 columns]

## Handling missing data

### Identify the missing values

#### Find the variables with NaN, their proportion of NaN, and dtype

In [0]:
# Define a function to check variables with NaN
def nan_checker(df):
  '''
  The function checks for NaN
  Parameter df: dataframe
  Returns the dataframe of variables with NaN, their proportion of NaN, and dtype
  '''

  # Get the variables with NaN, their proportion of NaN and dtype
  df_nan = pd.DataFrame([[var, df[var].isna().sum() / df.shape[0], df[var].dtype]
                         for var in df.columns if df[var].isna().sum() > 0],
                        columns = ['var', 'proportion', 'dtype'])
  
  # Sort df_nan in descending order of the proportion of NaN
  df_nan = df_nan.sort_values(by = 'proportion', ascending = False)

  return df_nan

In [52]:
# Call the function nan_checker on df
df_nan = nan_checker(df)

# Print df_nan
df_nan.reset_index(drop=True)

var  proportion   dtype
0             weight    0.983285  object
1  medical_specialty    0.481897  object
2         payer_code    0.340400  object
3               race    0.025486  object
4             diag_3    0.016421  object
5             diag_2    0.004726  object
6             diag_1    0.000165  object

There is no missing values that has numerical datatype. Variables 'weight', 'medical_specialty', and 'payer_code' have large proportions of NaN and they are object datatype, we will remove these three columns.

In [53]:
# Remove columns 'weight', 'medical_specialty', and 'payer_code'
df = df.drop(columns= ['weight', 'medical_specialty', 'payer_code'])

# Print the first 5 rows of df
df.head()

race  gender      age  ...  diabetesMed  readmitted  treatments
1  Caucasian  Female  [10-20)  ...          Yes         >30     insulin
3  Caucasian    Male  [30-40)  ...          Yes          NO     insulin
4  Caucasian    Male  [40-50)  ...          Yes          NO          io
5  Caucasian    Male  [50-60)  ...          Yes         >30     insulin
6  Caucasian    Male  [60-70)  ...          Yes          NO          io

[5 rows x 20 columns]

In [54]:
# Print the dimension of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns = ['# rows', '# columns'])

# rows  # columns
0   54383         20

## Encoding the data

In [55]:
# Print the unique dtypes of variables in df
pd.DataFrame(df.dtypes.unique(), columns = ['dtype'])

dtype
0  object
1   int64

### Identify Categorical Variables that have object as dtype


In [0]:
# Define a function to check for variables with object as dtype
def cat_var_checker(df):
  """
  The function checks for categorical variables
  Parameter df: dataframe

  Returns the dataframe of categorical variables and their number of unique value
  """

  # Get the dataframe of categorical variables and their number of unique value
  df_cat = pd.DataFrame([[var, df[var].nunique(dropna=False)] 
                        for var in df.columns if df[var].dtype == 'object'],
                        columns = ['var', 'nunique'])
  
  # Sort df_cat in descending order of the number of unique value
  df_cat = df_cat.sort_values(by='nunique', ascending=False).reset_index(drop=True)

  return df_cat

In [57]:
# Call the function on df
df_cat = cat_var_checker(df)

# Print df_cat
df_cat

var  nunique
0          diag_3      723
1          diag_2      666
2          diag_1      648
3             age       10
4            race        6
5   max_glu_serum        4
6       A1Cresult        4
7      readmitted        3
8          gender        2
9          change        2
10     treatments        2
11    diabetesMed        1

### Remove categorical features with large number of categories in df

In [0]:
# Remove diag_3, diag_2, diag_1 because they have large number of uniques in df.
# Another reason is they don't contribute to the outcome of the predictions of either readmitted or treatments.
df = df.drop(columns = ['diag_1', 'diag_2', 'diag_3'])

### Imputing NaN with mode


In [0]:
# Imputing NaN with mode
df['race'].fillna(df['race'].mode()[0], inplace=True)

### Group patients readmitted within and after 30 days as 1 group, label as 1 for readmitted. Label patients without readmitted as 0. 

In [0]:
# Label values from 'readmitted' column as follow: 0 for patients with no readmission, 1 for patients with admission within or after 30 days. 
# Therefore, 'NO' will be labeled as 0, '<30' and '>30' will be labeled as 1.
df['readmitted'] = df['readmitted'].replace(['NO', '<30', '>30'], [0,1,1])

In [62]:
# Print the dimesion of df
pd.DataFrame([[df.shape[0], df.shape[1]]], columns = ['# rows', '# columns'])

# rows  # columns
0   54383         17

In [63]:
# Print the first 5 rows of df
df.head()

race  gender      age  ...  diabetesMed  readmitted  treatments
1  Caucasian  Female  [10-20)  ...          Yes           1     insulin
3  Caucasian    Male  [30-40)  ...          Yes           0     insulin
4  Caucasian    Male  [40-50)  ...          Yes           0          io
5  Caucasian    Male  [50-60)  ...          Yes           1     insulin
6  Caucasian    Male  [60-70)  ...          Yes           0          io

[5 rows x 17 columns]

# EDA

# Predicting readmission

## Getting the name of the target

In [0]:
target_readmitted = 'readmitted'

## Encoding the categorical features using pandas.get_dummies

In [0]:
import numpy as np
# One-hot-encode the categorical features in the combined data
df = pd.get_dummies(df, columns = np.setdiff1d(np.intersect1d(df.columns, df_cat['var']), [target_readmitted]))

## Encoding the Categorical Target using sklearn.LabelEncoder

In [0]:
from sklearn.preprocessing import LabelEncoder

# The LabelEncoder
le = LabelEncoder()

# Encode the categorical target in the combined data
df[target_readmitted] = le.fit_transform(df[target_readmitted].astype(str))

## Dividing df into training and testing sets

In [0]:
from sklearn.model_selection import train_test_split
# Divide the training data into training (80%) and test (20%) using sklearn train_test_split with random_state=42 and stratify
df_train, df_test = train_test_split(df, train_size = 0.8, random_state = 42, stratify = df[target_readmitted])

# Reset the index
df_train, df_test = df_train.reset_index(drop=True), df_test.reset_index(drop=True)

## Dividing df_train into training and validation sets

In [0]:
from sklearn.model_selection import train_test_split
# Divide the training data into training (80%) and test (20%) using sklearn train_test_split with random_state=42 and stratify
df_train, df_valid = train_test_split(df_train, train_size = 0.8, random_state = 42, stratify = df_train[target_readmitted])

# Reset the index
df_train, df_valid = df_train.reset_index(drop=True), df_valid.reset_index(drop=True)

In [69]:
# Print the dimesion of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns = ['# rows', '# columns'])

# rows  # columns
0   34804         39

In [70]:
# Print the dimension of df_valid
pd.DataFrame([[df_valid.shape[0], df_valid.shape[1]]], columns = ['# rows', '# columns'])

# rows  # columns
0    8702         39

In [71]:
# Print the dimension of df_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns = ['# rows', '# columns'])

# rows  # columns
0   10877         39

## Getting the name of the features

In [0]:
# Get the name of features
features = np.setdiff1d(df.columns, [target_readmitted])

## Getting the feature matrix and target vector

In [0]:
# Get the feature matrix
X_train = df_train[features].to_numpy()
X_valid = df_valid[features].to_numpy()
X_test = df_test[features].to_numpy()

# Get the target vector
y_train = df_train[target_readmitted].astype(int).to_numpy()
y_valid = df_valid[target_readmitted].astype(int).to_numpy()

## Scaling the data

In [0]:
# Standardize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

# The StandardScaler
ss = StandardScaler()

# Standardize the training data
X_train = ss.fit_transform(X_train)

# Standardize the validation data
X_valid = ss.transform(X_valid)

# Standardize the testing data
X_test = ss.transform(X_test)

## Hyperparameter Tuning and Model Selection

The models we are using include Logistic Regression, Decision Tree, Random Forest Classification, Histogram-based Gradient Boost Classifier, and XGBoost Classifier.

### Create the dictionary of the models
In the dictionary:


*   the key is the acronym of the model
*   the value is the model




In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

models = {'lr': LogisticRegression(class_weight='balanced', random_state=42),
          'dtc': DecisionTreeClassifier(class_weight='balanced', random_state=42),
          'rfc': RandomForestClassifier(class_weight='balanced', random_state=42),
          'hgbc': HistGradientBoostingClassifier(random_state=42),
          'xgbc': XGBClassifier(seed=42)}

### Create the dictionary of the pipelines
In the dictionary:


*   the key is the acronym of the model
*   the value is the pipeline, which, for now, only includes the model



In [0]:
from sklearn.pipeline import Pipeline

pipes = {}

for acronym, model in models.items():
    pipes[acronym] = Pipeline([('model', model)])

### Getting the predefined split cross-validator

In [0]:
from sklearn.model_selection import PredefinedSplit

# Combine the feature matrix in the training and validation data
X_train_valid = np.vstack((X_train, X_valid))

# Combine the target vector in the training and validation data
y_train_valid = np.append(y_train, y_valid)

# Get the indices of training and validation data
train_valid_idxs = np.append(np.full(X_train.shape[0], -1), np.full(X_valid.shape[0], 0))

# The PredefinedSplit
ps = PredefinedSplit(train_valid_idxs)

### Create the dictionary of the parameter grids
In the dictionary:

*   the key is the acronym of the model
*   the value is the parameter grid of the model



In [0]:
param_grids = {}

#### The parameter grid for logistic regression
The hyperparameters we want to fine-tune are:

*   C
*   tol



In [0]:
# The grid for C
C_grids = [10**i for i in range(-6,3)]

# The grids for tol
tol_grids = [10 ** i for i in range(-6, -1)]

# Update param_grids
param_grids['lr'] = [{'model__C': C_grids,
                      'model__tol': tol_grids}]

#### The parameter grid for decision tree
The hyperparameters we want to fine-tune are:
- min_samples_split
- min_samples_leaf
- max_depth

In [0]:
# The grids for min_samples_split
min_samples_split_grids = [2, 30, 100]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 30, 100]

# The grids for max_depth
max_depth_grids = range(1, 11)

# Update param_grids
param_grids['dtc'] = [{'model__min_samples_split': min_samples_split_grids,
                       'model__min_samples_leaf': min_samples_leaf_grids,
                       'model__max_depth': max_depth_grids}]

#### The parameters grid for random forest
The hyperparameters we want to fine-tune are:

*   min_samples_split
*   min_samples_leaf



In [0]:
# The grids for min_samples_split
min_samples_split_grids = [2, 20, 100]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 20, 100]

# Implement me
# Update param_grids
param_grids['rfc'] = [{'model__min_samples_split': min_samples_split_grids,
                       'model__min_samples_leaf': min_samples_leaf_grids}]

#### The parameter grid for histogram-based gradient boosting
The hyperparameters we want to fine-tune are:

*   Learning_rate
*   min_samples_leaf



In [0]:
# The grids for learning_rate
learning_rate_grids = [10 ** i for i in range(-6, 3)]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 20, 100]

# Implement me
# Update param_grids
param_grids['hgbc'] = [{'model__learning_rate': learning_rate_grids,
                        'model__min_samples_leaf': min_samples_leaf_grids}]

#### The parameter grid for xgboost
The hyperparameters we want to fine-tune are:

*   eta
*   gamma
*   lambda





In [0]:
# The grids for eta
eta_grids = [10 ** i for i in range(-4, 1)]

# The grids for gamma
gamma_grids = [0, 10, 100]

# The grids for lambda
lambda_grids = [10 ** i for i in range(-4, 5)]

# Implement me
# Update param_grids
param_grids['xgbc'] = [{'model__eta': eta_grids,
                        'model__gamma': gamma_grids,
                        'model__lambda': lambda_grids}]

### Create the directory for cv results

In [0]:
import os

# Make directory
directory = os.path.dirname('./cv_results/')
if not os.path.exists(directory):
    os.makedirs(directory)

### Hyperparameter Tuning

In [0]:
from sklearn.model_selection import GridSearchCV

# The list of [best_score_, best_params_, best_estimator_] obtained by GridSearchCV
best_score_param_estimator_gs = []

for acronym in pipes.keys():
    # GridSearchCV
    gs = GridSearchCV(estimator=pipes[acronym],
                      param_grid=param_grids[acronym],
                      scoring='f1_micro',
                      n_jobs=2,
                      cv=ps,
                      return_train_score=True)
        
    # Fit the pipeline
    gs = gs.fit(X_train_valid, y_train_valid)
    
    # Update best_score_param_estimator_gs
    best_score_param_estimator_gs.append([gs.best_score_, gs.best_params_, gs.best_estimator_])
    
    # Sort cv_results in ascending order of 'rank_test_score' and 'std_test_score'
    cv_results = pd.DataFrame.from_dict(gs.cv_results_).sort_values(by=['rank_test_score', 'std_test_score'])
    
    # Get the important columns in cv_results
    important_columns = ['rank_test_score',
                         'mean_test_score', 
                         'std_test_score', 
                         'mean_train_score', 
                         'std_train_score',
                         'mean_fit_time', 
                         'std_fit_time',                        
                         'mean_score_time', 
                         'std_score_time']
    
    # Move the important columns ahead
    cv_results = cv_results[important_columns + sorted(list(set(cv_results.columns) - set(important_columns)))]

    # Write cv_results file
    cv_results.to_csv(path_or_buf='./cv_results/' + acronym + '.csv', index=False)

### Model Selection

In [86]:
# Sort best_score_param_estimator_gs in descending order of the best_score_
best_score_param_estimator_gs = sorted(best_score_param_estimator_gs, key=lambda x : x[0], reverse=True)

# Print best_score_param_estimator_gs
pd.DataFrame(best_score_param_estimator_gs, columns=['best_score', 'best_param', 'best_estimator'])

best_score  ...                                     best_estimator
0    0.634107  ...  (HistGradientBoostingClassifier(l2_regularizat...
1    0.631119  ...  ((DecisionTreeClassifier(ccp_alpha=0.0, class_...
2    0.629396  ...  (XGBClassifier(base_score=0.5, booster='gbtree...
3    0.626178  ...  (DecisionTreeClassifier(ccp_alpha=0.0, class_w...
4    0.622960  ...  (LogisticRegression(C=0.1, class_weight='balan...

[5 rows x 3 columns]